In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='1.12.0',image_scope='inference',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 520713654638.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving:1.12.0-cpu

In [ ]:
import scipy.sparse
from sagemaker import image_uris
image_uris.retrieve(framework='tensorflow',region='us-east-1',version='2.12.0',image_scope='training',instance_type='ml.c5.4xlarge')

In [ ]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin  763104351884.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
!docker build -t 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-housing -f Dockerfile.train .

In [ ]:
!docker build -t 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-test -f DockerfileTest.train .

In [4]:
!docker images

REPOSITORY                                                           TAG                        IMAGE ID       CREATED              SIZE
523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2   2.12.0-cpu-py310-housing   561383576411   About a minute ago   14.3GB


In [ ]:
!docker rmi -f $(docker images -aq)
!docker system prune -a -f --volumes

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-housing',
    role=role,
    instance_count=1,
    instance_type='local',
    hyperparameters=hyperparameters,
    output_path='file://{}/data/output'.format(os.getcwd()),
)

print('##### ESTIMATOR FIT STARTED')
print(os.getcwd())
input_path = 'file://{}/data/input/'.format(os.getcwd())
estimator.fit(input_path+"train.csv")
print('##### ESTIMATOR FIT COMPLETED')

In [ ]:
!tar -xzf data/output/output.tar.gz

In [ ]:
!SM_MODEL_DIR="./data/output" python main.py

In [ ]:
!tar -xzf ./data/output/model.tar.gz

In [5]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310-housing"
echo "fullname: ${fullname} ######################"

aws ecr create-repository --repository-name "${image_name}"

aws ecr describe-repositories --repository-names "${image_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${image_name}" > /dev/null
fi

image_name: tensorflow-training ######################
account: 891377019371 ######################
region: us-east-1 ######################
fullname: 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing ######################
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-east-1:891377019371:repository/tensorflow-training",
        "registryId": "891377019371",
        "repositoryName": "tensorflow-training",
        "repositoryUri": "891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training",
        "createdAt": 1730737511.505,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [6]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [7]:
!docker tag 523094669511.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training-2:2.12.0-cpu-py310-housing 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing 

In [11]:
!docker push 891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing

The push refers to repository [891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training]

36af527f: Preparing 
aff64f19: Preparing 
1892ae85: Preparing 
ff119fbf: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
80a2a6eb: Preparing 
a0538fa4: Preparing 
6d109c54: Preparing 
f0a068df: Preparing 
443b7673: Preparing 
37a36861: Preparing 
ecb4bd81: Preparing 
34b60c81: Preparing 
07d5d0ab: Preparing 
e6c8e003: Preparing 
b4f6696: Waiting g 
c6c24f9: Waiting g 
892ae85: Pushed   8.212GB/8.178GBPushing    333MB/443.1MBPushing    611MB/8.178GBPushing  280.9MB/285.6MBPushing  1.901GB/8.178GBPushing  2.867GB/8.178GBPushing  3.775GB/8.178GBPushing  4.876GB/8.178GBPushing  5

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='891377019371.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-housing',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://itsarghisdata/data/output'.format(os.getcwd())
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://itsarghisdata/data/input/train.csv'.format(os.getcwd()))
print('##### ESTIMATOR FIT COMPLETED')



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: tensorflow-training-2024-11-04-22-28-57-496


##### ESTIMATOR FIT STARTED
2024-11-04 22:28:59 Starting - Starting the training job...